## Dependencies

Install dependencies not available on Google Collab.
Collab provides numpy, pandas, sklearn, tensorflow, scipy, etc. (see requirements.txt)

In [6]:
!pip install pinard
!pip install scikeras

## Benchmark details

The results aggregate the combination of the following trainings configurations:
- estimation configuration: [regression, classification]
- datasets configurations: [Single Train, Cross validation with 5 folds and 2 repeats, Augmented Single Train]
- preprocessing configuration: [flat spectrum, savgol, haar, [small set], [big_set]]
- models: 
   - for all configuration: BACON, BACON-VG, DECON, PLS(components from 1 to 100), XGBoost, LW-PLS
   - for single train + small_set : Stack > [ BACON, BACON-VG, DECON, PLS(components from 1 to 100), XGBoost, LW-PLS,
   f_PLSRegression,f_AdaBoostRegressor,f_BaggingRegressor,f_ExtraTreesRegressor, f_GradientBoostingRegressor,f_RandomForestRegressor,
   f_ARDRegression,f_BayesianRidge,f_ElasticNet,f_ElasticNetCV,f_HuberRegressor, f_LarsCV,f_LassoCV,f_Lasso,f_LassoLars,f_LassoLarsCV,
   f_LassoLarsIC,f_LinearRegression,f_OrthogonalMatchingPursuit,f_OrthogonalMatchingPursuitCV, f_PassiveAggressiveRegressor,f_RANSACRegressor,
   f_Ridge,f_RidgeCV,f_SGDRegressor,f_TheilSenRegressor,f_GaussianProcessRegressor,f_KNeighborsRegressor, f_Pipeline,f_MLPRegressor,f_LinearSVR,
   f_NuSVR,f_SVR,f_DecisionTreeRegressor,f_ExtraTreeRegressor,f_KernelRidge,f_XGBRegressor]

We perform training in 2 steps, (1) data transformation and (2) training because the sklearn pipeline does not use test data natively.
To change with pinard update in the future.

In [1]:
### FAST GPU RESET ####
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [68]:
## Browse path and launch benchmark for every folders
%load_ext autoreload
%autoreload 2

from pathlib import Path
from preprocessings import preprocessing_list

from benchmark_loop import benchmark_dataset

rootdir = Path('data/regression')
folder_list = [f for f in rootdir.glob('**/*') if f.is_dir()]

SEED = ord('D') + 31373

import preprocessings
import regressors
import pinard.preprocessing as pp
from pinard import augmentation, model_selection
from sklearn.cross_decomposition import PLSRegression
from xgboost import XGBRegressor
import sys
import os.path

def str_to_class(classname):
    return getattr(sys.modules['pinard.preprocessing'], classname)

# print(str_to_class('SavitzkyGolay'))




def get_dataset_list(path):
    datasets = []
    for r, d, _ in os.walk(path):
        for folder in d:
            path = os.path.join(r, folder)
            if os.path.isdir(path):
                if len(datasets) < 3:
                    datasets.append(str(path))
    return datasets

split_configs = [
    None,
    {'test_size':None, 'method':"random", 'random_state':SEED},
    {'test_size':None, 'method':"stratified", 'random_state':SEED, 'n_bins':5},
    {'test_size':None, 'method':"kennard_stone", 'random_state':SEED, 'metric':"euclidean", 'pca_components':None},
]

augmentations = [
    None,
    [(2, augmentation.Rotate_Translate()),
    (2, augmentation.Random_X_Operation()),
    (1, augmentation.Random_Spline_Addition()),]
]

preprocessings_list = [
    # preprocessings.id_preprocessing(),
    [('haar', pp.Haar()), ('savgol', pp.SavitzkyGolay())],
    preprocessings.decon_set(),
]

models = [
    (regressors.ML_Regressor(XGBRegressor), {"n_estimators":200, "max_depth":50, "seed":SEED}),
    (regressors.VGG_1D(), {'batch_size':256, 'epoch':10, 'verbose':0, 'optimizer':'Adam', 'loss':'mse'}),
]

cv_configs = [
    None,
    {'n_splits':3, 'n_repeats':1},
    {'n_splits':5, 'n_repeats':2},
]

# import os
folder = "data/regression"
folders = get_dataset_list(folder)

benchmark_size = len(folders) * len(split_configs) * len(cv_configs) * len(augmentations) * len(preprocessings_list) * len(models)
print("Benchmarking", benchmark_size, "runs.")

# benchmark_dataset(folders, split_configs, cv_configs, augmentations, preprocessings_list, models, SEED,)
benchmark_dataset(["data/regression/ALPINE_Calpine_424_Murguzur_RMSE1.36"], split_configs, cv_configs, augmentations, preprocessings_list, models, SEED,)


# (preprocessing_list, nn_run, nn_cv, ml_single, ml_cv)

# benchmark_dataset("data/regression/ALPINE_Calpine_424_Murguzur_RMSE1.36", SEED, preprocessing_list(), 200)
# benchmark_dataset("data/regression/Cassava_TBC_3556_Davrieux_RMSE1.02", SEED, preprocessing_list(), augment=True)
# benchmark_dataset("data/regression/LUCAS_SOCgrassland_4096_Nocita_RMSE7.2", SEED, preprocessing_list(), 20, augment=False)
# benchmark_dataset("data/regression/ALPINE_Calpine_424_Murguzur_RMSE1.36", SEED, preprocessing_list(), 100, augment=False)
# benchmark_dataset("data/regression/Cassava_TBC_3556_Davrieux_RMSE1.02", SEED, preprocessing_list(), 20, augment=False)
# benchmark_dataset("data/regression/Meat_Fa.33", SEED, preprocessing_list(), 100, augment=False)

# for folder in folder_list:
    # # print(ord(str(folder)[17]), ord('A'), ord('M'))
    # if ord(str(folder)[16]) < ord("L") or ord(str(folder)[16]) > ord("M"):
    #     continue
    # benchmark_dataset(folder, SEED, preprocessing_list(), 20, augment=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Benchmarking 288 runs.
========== ALPINE_CALPINE_424_MURGUZUR ========== (272, 2151) (272, 1) (152, 2151) (152, 1)
VGG_1D-NoSpl-NoCV-Fold_1(1)-NoAug-PP_2_-3462-31441_22-12-14_03-39-11 (272, 2151) (272, 1) (152, 2151, 2) (152, 1)
--- Trainable: 1479969 - untrainable: 0.0 > 1479969.0


Epoch: 0 > RMSE: 17.768942 ( 1.36 ) - R²: -38.521912791516804  val_loss 0.78271484375
Epoch: 1 > RMSE: 16.57503 ( 1.36 ) - R²: -33.389306668860605  val_loss 0.68115234375
Epoch: 2 > RMSE: 11.534788 ( 1.36 ) - R²: -15.654595046752899  val_loss 0.329833984375
Epoch: 3 > RMSE: 5.4681754 ( 1.36 ) - R²: -2.742826858598239  val_loss 0.0740966796875
Epoch: 7 > RMSE: 3.540156 ( 1.36 ) - R²: -0.5687711427601276  val_loss 0.0310211181640625
Epoch: 8 > RMSE: 3.497179 ( 1.36 ) - R²: -0.5309130497428247  val_loss 0.0303497314453125
Saved best 0.0303 at epoch 8
3.497179  ( 1.36 ) in 00:00:04
xgboost.sklearn.XGBRegressor-NoSpl-NoCV-Fold_1(1)-NoAug-PP_2_-3462-31441_22-12-14_03-39-15 (272, 2151) (272, 1) (152, 4302) (152, 1)
(self, *, objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'reg:squarederror', **kwargs: Any) -> None
2.4760873  ( 1.36 ) in 00:00:02


In [ ]:
split_list = [
    ({"method": "random", "test_size": 0.25, "random_state": 42}, 0),
    (
        {
            "method": "k_mean",
            "test_size": 0.25,
            "random_state": 42,
            "metric": "canberra",
        },
        1,
    ),
    (
        {
            "pca_components": 4,
            "method": "k_mean",
            "test_size": 0.25,
            "random_state": 42,
            "metric": "canberra",
        },
        2,
    ),
    ({"method": "kennard_stone", "test_size": 0.25, "random_state": 42}, 3),
    (
        {
            "method": "kennard_stone",
            "test_size": 0.25,
            "random_state": 42,
            "metric": "correlation",
            "pca_components": 8,
        },
        4,
    ),
    (
        {
            "method": "kennard_stone",
            "test_size": 0.25,
            "random_state": 42,
            "metric": "correlation",
        },
        5,
    ),
    ({"method": "spxy", "test_size": 0.25, "random_state": 42}, 6),
    ({"method": "spxy", "test_size": 0.25, "random_state": 42, "pca_components": 2}, 7),
    (
        {"method": "spxy", "test_size": 0.25, "random_state": 42, "metric": "canberra"},
        8,
    ),
    ({"method": "stratified", "test_size": 0.25, "random_state": 42}, 9),
    ({"method": "stratified", "test_size": 0.25, "random_state": 42, "n_bins": 4}, 10),
    ({"method": "circular", "test_size": 0.25, "random_state": 42}, 11),
]

In [8]:
# config = {
#     'augmentation':[],
#     'preprocessing':[],
#     'runs':[
#         {
#             'model':'PLS',
#         }
#     ]
# }


# def parse_json(config, name=''):
#     collection = []
#     if isinstance(config, list):
        
#         pass
#     elif isinstance(config, dict):
#         pass
    
#     return name, collection
a = [(5,6)]

for b,c  in iter(a):
    print(b,c )

5 6
